In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import seaborn as sn
from math import floor, ceil
from pylab import rcParams
from sklearn import preprocessing
import time
import matplotlib.pyplot as plt

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
features = np.loadtxt('vec_mat_features_icex_src_0.01train.csv',delimiter=",")
labels_unstand = np.loadtxt('vec_mat_rlabels_icex_src_0.01train.csv',delimiter=",")

X_train = preprocessing.scale(features)
    
labels_unstand = labels_unstand.ravel()
#y_train,mu,sigma_labels = std_y(labels_unstand)

y_train = labels_unstand

In [ ]:
X_test = np.loadtxt('vec_mat_features_icex_src_test6.csv',delimiter=",")
temp_ytest = np.loadtxt('vec_mat_rlabels_icex_src_test6.csv',delimiter=",")
y_test= []

X_test = preprocessing.scale(X_test)

temp_ytest = temp_ytest.ravel()
#y_test = (temp_ytest - mu)/sigma_labels
y_test = temp_ytest


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
drate = 0.5
n_node = 256
batch_size = 128
activation = tf.nn.sigmoid
loss='mean_squared_error'

model = keras.Sequential([
    keras.layers.Dense(n_node, activation),
    keras.layers.Dropout(drate),
    keras.layers.Dense(n_node, activation),
    keras.layers.Dropout(drate),
    keras.layers.Dense(n_node, activation),
    keras.layers.Dropout(drate),
    keras.layers.Dense(n_node, activation),
    keras.layers.Dropout(drate),
    keras.layers.Dense(n_node, activation),
    keras.layers.Dense(1, keras.activations.linear)])

lr = 0.0001
optimizer = keras.optimizers.Adam(lr)

model.compile(optimizer,loss, metrics=['mape'])

t = time.time()
infdb = model.fit(X_train, y_train, batch_size,verbose=True, epochs=500, validation_split=0.0, validation_data=(X_test,y_test), shuffle=True)
elapsed = time.time() - t
print(elapsed)

In [ ]:
#Testing

test_loss, test_acc = model.evaluate(X_test,y_test)
train_loss, train_acc = model.evaluate(X_train,y_train)

print('Training mape:', train_acc)
print('Test mape:', test_acc)

predictions = model.predict(X_test)

In [ ]:
#diff = np.abs((y_pred*sigma_labels+mu)-(y_test*sigma_labels+mu))
diff = (np.transpose(predictions))-(y_test)
error = diff[diff>1]
percent_correct = (len(y_test)-len(error))/len(y_test)
print(percent_correct)

In [ ]:
epoch = np.arange(1,501)
fig1 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('SNR = INF')
plt.plot(epoch, infdb.history['loss'], label = 'Training')
plt.plot(epoch, infdb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('MSE Loss')
plt.axis([1, 500, 0, 1000])
plt.subplot(1,2,2)
plt.plot(epoch, np.array(infdb.history['mean_absolute_percentage_error']),label = 'Training')
plt.plot(epoch, np.array(infdb.history['val_mean_absolute_percentage_error']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('MAPE')
plt.grid()
plt.axis([1, 500, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
#fig1.savefig('rfnnp2_INFdb.png')
'''
fig2 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('SNR = 10dB')
plt.plot(epoch, tendb.history['loss'], label = 'Training')
plt.plot(epoch, tendb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('MSE Loss')
plt.axis([1, 500, 0, 1000])
plt.subplot(1,2,2)
plt.plot(epoch, np.array(tendb.history['mean_absolute_percentage_error']),label = 'Training')
plt.plot(epoch, np.array(tendb.history['val_mean_absolute_percentage_error']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('MAPE')
plt.grid()
plt.axis([1, 500, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig2.savefig('rfnnp2_10db.png')

fig3 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('SNR = 5dB')
plt.plot(epoch, fivedb.history['loss'], label = 'Training')
plt.plot(epoch, fivedb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('MSE Loss')
plt.axis([1, 500, 0, 1000])
plt.subplot(1,2,2)
plt.plot(epoch, np.array(fivedb.history['mean_absolute_percentage_error']),label = 'Training')
plt.plot(epoch, np.array(fivedb.history['val_mean_absolute_percentage_error']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('MAPE')
plt.grid()
plt.axis([1, 500, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig3.savefig('rfnnp2_5db.png')

fig4 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('SNR = 0dB')
plt.plot(epoch, zerodb.history['loss'], label = 'Training')
plt.plot(epoch, zerodb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('MSE Loss')
plt.axis([1, 500, 0, 1000])
plt.subplot(1,2,2)
plt.plot(epoch, np.array(zerodb.history['mean_absolute_percentage_error']),label = 'Training')
plt.plot(epoch, np.array(zerodb.history['val_mean_absolute_percentage_error']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('MAPE')
plt.grid()
plt.axis([1, 500, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig4.savefig('rfnnp2_0db.png')

fig5 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('SNR = -5dB')
plt.plot(epoch, mfivedb.history['loss'], label = 'Training')
plt.plot(epoch, mfivedb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('MSE Loss')
plt.axis([1, 500, 0, 1000])
plt.subplot(1,2,2)
plt.plot(epoch, np.array(mfivedb.history['mean_absolute_percentage_error']),label = 'Training')
plt.plot(epoch, np.array(mfivedb.history['val_mean_absolute_percentage_error']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('MAPE')
plt.grid()
plt.axis([1, 500, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig5.savefig('rfnnp2_m5db.png')

fig6 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('SNR = -10dB')
plt.plot(epoch, mtendb.history['loss'], label = 'Training')
plt.plot(epoch, mtendb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('MSE Loss')
plt.axis([1, 500, 0, 1000])
plt.subplot(1,2,2)
plt.plot(epoch, np.array(mtendb.history['mean_absolute_percentage_error']),label = 'Training')
plt.plot(epoch, np.array(mtendb.history['val_mean_absolute_percentage_error']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('MAPE')
plt.grid()
plt.axis([1, 500, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig6.savefig('rfnnp2_m10db.png')
'''

In [ ]:
#sort output for plotting

inds = y_test.argsort()
sorted_y_test = y_test[inds]

sorted_y_pred = np.squeeze(predictions)[inds]


In [ ]:
#Plotting
fig1 = plt.figure(figsize = (20,14))
plt.scatter(sorted_y_test,sorted_y_pred)
plt.axis([0, 50, 0, 50])
plt.ylabel('Predicted Distance (km)',fontsize=20)
plt.xlabel('Actual Distance (km)',fontsize=20)
plt.title('SNR = INF',fontsize=20)
plt.grid()
#fig1.savefig('rfnnp2_scat_infdb.png')